#Read multiple files from nested folder

In [0]:
#create multiple files in nested folder
dbutils.fs.put("/information/person1.csv","""id,name,role
1,sree,engg
2,sree,DataEng
3,Sreejish,Engineering
""")
dbutils.fs.put("/information/2022/person.csv","""id,name,role
1,sree,engg
2,sree,DataEng
3,Sreejish,Engineering
""")


In [0]:
%fs ls information/

In [0]:
#to read multiple file use recursiveFileLookup option
from pyspark.sql.functions import input_file_name # to get file name
df = spark.read.format("csv").option("recursiveFileLookup",True).option("header",True).load("/information")
#get file paths to find rows from each file
df.withColumn("filename",input_file_name()).display()

Get number of rows from each file in a spark dataframe

In [0]:
%fs ls /information

In [0]:
#from previous dataframe get rows in each file
from pyspark.sql.functions import input_file_name
df=df.withColumn("file_name",input_file_name())
df.groupBy("file_name").count().display()

get number of rows at each partition level

In [0]:
%fs ls /databricks-datasets/COVID/USAFacts


In [0]:
df_covid = spark.read.csv("/databricks-datasets/COVID/USAFacts/*.csv",header=True)

In [0]:
#partition using rdd
df_covid.rdd.getNumPartitions()

In [0]:
#use spark_partition_id to find partition
from pyspark.sql.functions import spark_partition_id
df_covid.select(spark_partition_id().alias("part_id")).groupBy("part_id").count().show()

In [0]:
#if use repartition it will distribute and recreate partition equally
df_covid.repartition(6).select(spark_partition_id().alias("part_id")).groupBy("part_id").count().orderBy("part_id").show()